In [1]:
import pandas as pd
import os
import numpyro
from refit_fvs.data.load import fia_for_diameter_growth_modeling
from refit_fvs.models.fit import fit_wykoff, fit_simpler_wykoff

In [2]:
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.75"
numpyro.set_platform("gpu")
numpyro.set_host_device_count(3)

# Select species
Identify species codes and the FVS variant from which bark ratio coefficients should be drawn.

In [3]:
FVS_ALPHA = "BO"
BARK_VAR = "CA"
FIA_SPCD = 818

In [4]:
BARK_COEFS = "../../data/raw/fvs_barkratio_coefs.csv"
bark = pd.read_csv(BARK_COEFS).set_index(["FIA_SPCD", "FVS_VARIANT"])

bark_b0, bark_b1, bark_b2 = bark.loc[FIA_SPCD, BARK_VAR][["BARK_B0", "BARK_B1", "BARK_B2"]]
bark_b0, bark_b1, bark_b2

(-0.26824, 0.95767, 1.0)

# Model-fitting parameters
These don't usually need to be updated when fitting a new species unless you want to add more chains or samples per chain.

In [5]:
NUM_CYCLES = 2
LOC_RANDOM = True
PLOT_RANDOM = True
CHECKPOINT_DIR = f"../../models/maicf/{FVS_ALPHA}/"
NUM_WARMUP = 1000
NUM_SAMPLES = 1000
NUM_CHAINS = 1
CHAIN_METHOD = "parallel"
NUM_BATCHES = 1
SEED = 42
PROGRESS_BAR = True
OVERWRITE = False

In [6]:
data, factors = fia_for_diameter_growth_modeling(
    path="../../data/interim/FIA_remeasured_trees_for_training.csv",
    filter_spp=[FIA_SPCD]
)
obs_variants, obs_locations, obs_plots = factors
data.describe()

,LOCATION,PLOT_ID,FIA_SPCD,MAICF,SLOPE,ASPECT,ELEV,DBH,CR,CR_NEXT,...,INC5YR,INC10YR,MEASYEAR,MEASMON,MEASYEAR_NEXT,MEASMON_NEXT,MEAS_INTERVAL,VAR_IDX,LOC_IDX,PLOT_IDX
count,1750.000000,1.750000e+03,1750.0,1750.000000,1750.000000,1750.000000,1750.000000,1750.000000,1750.000000,1750.000000,...,0.0,0.0,1750.000000,1750.000000,1750.000000,1750.000000,1750.0,1750.000000,1750.000000,1750.000000
mean,526.341143,1.020847e+11,818.0,91.021797,0.378926,3.079868,33.443429,11.165657,0.416160,0.401840,...,NaN,NaN,2004.683429,7.990286,2014.683429,6.873714,10.0,2.492000,11.561714,213.757143
std,44.944431,1.135046e+11,0.0,61.696896,0.199458,1.863211,13.911258,7.202577,0.166822,0.168117,...,NaN,NaN,2.554004,2.142819,2.554004,2.085520,0.0,2.290931,5.408675,119.655556
min,503.000000,6.010235e+10,818.0,4.080000,0.000000,0.000000,3.000000,5.000000,0.010000,0.010000,...,NaN,NaN,2001.000000,3.000000,2011.000000,3.000000,10.0,0.000000,0.000000,0.000000
25%,508.000000,6.020936e+10,818.0,50.042503,0.200000,1.658063,23.000000,6.400000,0.300000,0.300000,...,NaN,NaN,2002.000000,6.000000,2012.000000,5.000000,10.0,0.000000,9.000000,107.000000
50%,511.000000,6.030340e+10,818.0,94.337446,0.400000,3.106686,33.000000,8.400000,0.400000,0.400000,...,NaN,NaN,2004.000000,8.000000,2014.000000,7.000000,10.0,4.000000,14.000000,217.000000
75%,516.000000,6.050398e+10,818.0,124.201204,0.520000,4.699299,43.000000,12.875000,0.500000,0.500000,...,NaN,NaN,2007.000000,10.000000,2017.000000,9.000000,10.0,5.000000,15.000000,319.750000
max,712.000000,4.102034e+11,818.0,523.332582,0.970000,6.283185,73.000000,50.600000,0.950000,0.950000,...,NaN,NaN,2009.000000,12.000000,2019.000000,11.000000,10.0,5.000000,20.000000,416.000000


In [7]:
for tree_comp in ['bal', 'relht', 'ballndbh']:
    for stand_comp in ['ba', 'lnba', 'ccf']:
        for pooling in ['unpooled', 'partial']:
            model_name = f"wykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

wykoff_unpooled_mixed_bal-ba
wykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [15:04<00:00,  2.21it/s, 255 steps of size 1.86e-02. a


Done.
Samples saved at ../../models/maicf/BO//wykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/BO//wykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
wykoff_partial_mixed_bal-ba
wykoff_partial_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [23:03<00:00,  1.45it/s, 255 steps of size 1.91e-02. a


Done.
Samples saved at ../../models/maicf/BO//wykoff_partial_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/BO//wykoff_partial_mixed_bal-ba_batch00_mcmc.pkl
wykoff_unpooled_mixed_bal-lnba
wykoff_unpooled_mixed_bal-lnba_batch00


...

Done.
Samples saved at ../../models/maicf/BO//wykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/BO//wykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
wykoff_unpooled_mixed_ballndbh-ccf
wykoff_unpooled_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [12:59<00:00,  2.57it/s, 255 steps of size 2.25e-02. a


Done.
Samples saved at ../../models/maicf/BO//wykoff_unpooled_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/BO//wykoff_unpooled_mixed_ballndbh-ccf_batch00_mcmc.pkl
wykoff_partial_mixed_ballndbh-ccf
wykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [23:11<00:00,  1.44it/s, 255 steps of size 1.97e-02. a


Done.
Samples saved at ../../models/maicf/BO//wykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/BO//wykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl


In [8]:
for pooling in ['unpooled', 'partial']:
    for tree_comp in ['bal', 'relht', 'ballndbh']:
        for stand_comp in ['ba', 'lnba', 'ccf']:
            model_name = f"simplerwykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_simpler_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

simplerwykoff_unpooled_mixed_bal-ba
simplerwykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [11:27<00:00,  2.91it/s, 255 steps of size 1.66e-02. a


Done.
Samples saved at ../../models/maicf/BO//simplerwykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/BO//simplerwykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-lnba
simplerwykoff_unpooled_mixed_bal-lnba_batch00


sample: 100%|█| 2000/2000 [11:12<00:00,  2.98it/s, 255 steps of size 2.06e-02. a


Done.
Samples saved at ../../models/maicf/BO//simplerwykoff_unpooled_mixed_bal-lnba_batch00.pkl
MCMC saved at ../../models/maicf/BO//simplerwykoff_unpooled_mixed_bal-lnba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-ccf
simplerwykoff_unpooled_mixed_bal-ccf_batch00


...

Done.
Samples saved at ../../models/maicf/BO//simplerwykoff_partial_mixed_ballndbh-ba_batch00.pkl
MCMC saved at ../../models/maicf/BO//simplerwykoff_partial_mixed_ballndbh-ba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-lnba
simplerwykoff_partial_mixed_ballndbh-lnba_batch00


sample: 100%|█| 2000/2000 [17:46<00:00,  1.88it/s, 255 steps of size 2.01e-02. a


Done.
Samples saved at ../../models/maicf/BO//simplerwykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/BO//simplerwykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-ccf
simplerwykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [18:07<00:00,  1.84it/s, 255 steps of size 1.70e-02. a


Done.
Samples saved at ../../models/maicf/BO//simplerwykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/BO//simplerwykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl
